<a href="https://colab.research.google.com/github/ArkadiyReydman/Amazon-Walmart-Etsy-Price-Tracker/blob/master/%5Bfeb27%5D_deep_researcher_gemini%2Btvly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-generativeai
!pip install langchain
!pip install langchain-community
!pip install tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.3 MB/s eta 0:00:00


In [ ]:
import os
import time
from IPython.display import Markdown, display
import google.generativeai as genai
from langchain_community.tools.tavily_search import TavilySearchResults
from google.colab import userdata

# Initialize components
def initialize_components():
    # Configure Gemini
    genai_api_key = userdata.get('new_gemini')
    genai.configure(api_key=genai_api_key)
    gemini_model = genai.GenerativeModel("gemini-2.0-flash")

    # Initialize Tavily search - Set environment variable for Tavily
    tavily_api_key = userdata.get('tvly')
    if not tavily_api_key:
        raise ValueError("Tavily API key not found. Please add it to userdata with the key 'tvly'")

    # Set the API key as an environment variable which is how the wrapper expects it
    os.environ['TAVILY_API_KEY'] = tavily_api_key

    # Initialize Tavily with environment variable
    tavily_search = TavilySearchResults(max_results=5)

    return {
        'gemini_model': gemini_model,
        'tavily_search': tavily_search
    }

# Function to invoke Gemini model
def invoke_gemini(model, prompt, temperature=0.7, max_tokens=8192):
    generation_config = {
        "temperature": temperature,
        "top_p": 0.95,
        "top_k": 40,
        "max_output_tokens": max_tokens,
    }

    response = model.generate_content(prompt, generation_config=generation_config)
    return response.text

# Generate a research plan
def generate_research_plan(model, topic, focus_entities=None, data_points=None, special_instructions=None):
    print(f"Generating research plan for: {topic}")

    # Prepare focus entities section
    focus_entities_text = ""
    if focus_entities and len(focus_entities) > 0:
        focus_entities_text = "3. Specific information to look for regarding: " + ", ".join(focus_entities)

    # Prepare data points section
    data_points_text = "4. Important data points to include (market sizes, growth rates, trends, etc.)"
    if data_points and len(data_points) > 0:
        data_points_text += f", especially: {', '.join(data_points)}"

    # Add special instructions if provided
    special_instructions_text = ""
    if special_instructions:
        special_instructions_text = f"5. Special focus areas:\n{special_instructions}"

    prompt = f"""
    You are a research planning assistant. I need to create a thorough analysis report on the following topic:

    {topic}

    Please create a detailed research plan that includes:
    1. Key topics and subtopics to investigate
    2. Important questions to answer in each section
    {focus_entities_text}
    {data_points_text}
    {special_instructions_text}

    Structure your response as a comprehensive research plan with clear sections.
    """

    return invoke_gemini(model, prompt, temperature=0.7)

# Generate search queries based on the research plan
def generate_search_queries(model, topic, research_plan, focus_entities=None, data_points=None):
    print("Generating search queries...")

    # Prepare focus entities and data points texts
    focus_text = ""
    if focus_entities and len(focus_entities) > 0:
        focus_text = f"Pay special attention to these entities: {', '.join(focus_entities)}."

    data_text = ""
    if data_points and len(data_points) > 0:
        data_text = f"Make sure to create queries that will find these specific data points: {', '.join(data_points)}."

    prompt = f"""
    Based on this research topic and plan, generate 10-15 specific, effective search queries that will yield the most relevant and comprehensive information.

    Research Topic: {topic}

    Research Plan:
    {research_plan}

    {focus_text}
    {data_text}

    For each main section or question in the research plan, create 1-3 search queries that are:
    1. Specific enough to return relevant results
    2. Varied to cover different aspects of the topic
    3. Phrased to find current, data-rich sources
    4. Designed to find contrasting perspectives

    Format your response as a simple list of search queries, one per line, without numbering or additional text.
    """

    response = invoke_gemini(model, prompt, temperature=0.7)
    queries = [q.strip() for q in response.strip().split('\n') if q.strip()]
    return queries

# Perform search queries using Tavily
def search_topics(tavily_search, queries, search_depth="advanced"):
    print(f"Searching using {len(queries)} queries...")

    results = []
    for i, query in enumerate(queries):
        print(f"Searching ({i+1}/{len(queries)}): {query}")

        # Set search depth if supported
        if hasattr(tavily_search, 'search_depth'):
            tavily_search.search_depth = search_depth

        search_result = tavily_search.invoke({"query": query})

        # Format results
        formatted_results = {"results": []}
        for item in search_result:
            formatted_results["results"].append({
                "title": item.get("title", "No title"),
                "url": item.get("url", "No URL"),
                "content": item.get("content", "No content")[:1500]  # Trim content length
            })

        results.append({"query": query, "results": formatted_results})

        # Add a small delay between searches
        if i < len(queries) - 1:
            time.sleep(1)

    return results

# Analyze and synthesize search results
def synthesize_results(model, search_results, research_plan, topic, focus_entities=None):
    print("Synthesizing research findings...")

    # Prepare the content to send to Gemini
    search_content = ""

    for item in search_results:
        results_for_query = item['results'].get('results', [])
        if results_for_query:
            search_content += f"\nQuery: {item['query']}\n"
            for i, result in enumerate(results_for_query[:3]):  # Limit to top 3 results per query
                search_content += f"Result {i+1}: {result.get('title', 'No title')}\n"
                search_content += f"Source: {result.get('url', 'No URL')}\n"
                search_content += f"Content: {result.get('content', 'No content')}\n\n"

    # Prepare focus entities text if provided
    focus_entities_text = ""
    if focus_entities and len(focus_entities) > 0:
        focus_entities_text = f"Pay special attention to these entities and provide detailed analysis on each: {', '.join(focus_entities)}."

    prompt = f"""
    You are a research analyst specializing in comprehensive analysis. I will provide you with search results about "{topic}", along with a research plan. Your task is to synthesize this information into a comprehensive, data-driven report.

    Research Plan:
    {research_plan}

    Search Results:
    {search_content}

    Based on the search results and research plan, create a comprehensive, well-structured report that:
    1. Provides an overview of the topic
    2. Follows the structure outlined in the research plan
    3. Incorporates data and specific information from the search results
    4. Analyzes trends, patterns, and implications
    {focus_entities_text}

    Format the report with clear headings, subheadings, and bullet points where appropriate. Include specific data points from the search results and cite numbers when available.
    """

    return invoke_gemini(model, prompt, temperature=0.3, max_tokens=8192)

# Main research function
def run_research(topic, focus_entities=None, data_points=None, special_instructions=None, search_depth="advanced", output_file=None):
    """
    Run deep research on any topic using Gemini and Tavily

    Parameters:
    - topic (str): The main research topic
    - focus_entities (list): Specific companies, people, products, etc. to focus on
    - data_points (list): Specific data points to look for (revenue, market size, etc.)
    - special_instructions (str): Any special research instructions
    - search_depth (str): "advanced" or "basic" for search depth
    - output_file (str): Output filename (defaults to topic name + research_report.md)

    Returns:
    - The completed research report
    """
    # Initialize
    components = initialize_components()
    gemini_model = components['gemini_model']
    tavily_search = components['tavily_search']

    # Step 1: Generate a research plan
    research_plan = generate_research_plan(gemini_model, topic, focus_entities, data_points, special_instructions)
    display(Markdown("## Research Plan\n\n" + research_plan))

    # Step 2: Generate search queries
    search_queries = generate_search_queries(gemini_model, topic, research_plan, focus_entities, data_points)

    # Step 3: Perform searches based on the queries
    search_results = search_topics(tavily_search, search_queries, search_depth)

    # Step 4: Generate the report
    report = synthesize_results(gemini_model, search_results, research_plan, topic, focus_entities)

    # Display the final report
    display(Markdown(f"## Research Report: {topic}\n\n" + report))

    # Save the report
    file_name = output_file or f"{topic.lower().replace(' ', '_')[:50]}_research_report.md"
    with open(file_name, "w") as f:
        f.write(f"# Research Report: {topic}\n\n")
        f.write(report)

    print(f"Report saved to '{file_name}'")
    return report

# Example usage
if __name__ == "__main__":
    try:
        run_research(
            topic="The cultural impact of rubber ducks in modern society",
            focus_entities=["Programming rubber ducks", "Bathtub rubber ducks", "Giant rubber duck art installations"],
            data_points=["Annual sales figures", "Psychological benefits of rubber duck debugging"],
            special_instructions="Include analysis of how rubber ducks have evolved from children's bath toys to programming tools."
        )
    except Exception as e:
        print(f"Error running research: {e}")

Generating research plan for: The cultural impact of rubber ducks in modern society


## Research Plan

Okay, here's a detailed research plan for your analysis report on the cultural impact of rubber ducks in modern society.

**Research Plan: The Cultural Impact of Rubber Ducks in Modern Society**

**I. Introduction**

*   **Goal:**  Introduce the topic, establish the scope of the research, and provide a brief overview of the rubber duck's journey from bath toy to cultural icon.
*   **Key Topics:**
    *   Brief history of rubber ducks (early versions, materials, key manufacturers)
    *   Thesis statement outlining the multifaceted cultural impact of rubber ducks.
    *   Overview of the report's structure and key arguments.
*   **Important Questions:**
    *   What are the origins of the rubber duck as a toy?
    *   How has the rubber duck's design and manufacturing evolved over time?
    *   What is the central argument of this report regarding the rubber duck's cultural significance?
*   **Specific Information:**
    *   Early patents related to rubber duck toys.
    *   Key milestones in rubber duck design and production (e.g., the introduction of the squeaker, the use of PVC).
    *   Examples of the rubber duck's presence in popular culture (film, television, literature).

**II. The Rubber Duck as a Childhood Staple**

*   **Goal:** Explore the traditional role of rubber ducks as children's bath toys and their impact on early childhood development.
*   **Key Topics:**
    *   The rubber duck's role in bath time rituals.
    *   Sensory stimulation and developmental benefits for infants and toddlers.
    *   Nostalgia and childhood memories associated with rubber ducks.
    *   Marketing and branding of rubber ducks aimed at children.
*   **Important Questions:**
    *   How do rubber ducks contribute to a child's sensory experience during bath time?
    *   What are the psychological benefits of playing with rubber ducks in early childhood?
    *   How has the marketing of rubber ducks influenced their popularity among children?
    *   What are the common themes and emotions associated with rubber ducks in childhood memories?
*   **Specific Information:**
    *   **Bathtub Rubber Ducks:**
        *   Common designs and materials.
        *   Safety standards and regulations.
        *   Marketing campaigns targeting parents and children.
    *   Quotes and anecdotes about children's experiences with rubber ducks.
    *   Studies on the developmental benefits of bath time play.

**III. The Rubber Duck as a Programming Tool: Rubber Duck Debugging**

*   **Goal:** Examine the adoption of rubber ducks as a problem-solving tool in software development.
*   **Key Topics:**
    *   Origin and explanation of the "Rubber Duck Debugging" technique.
    *   Benefits of using rubber ducks for problem-solving and code review.
    *   Case studies and examples of rubber duck debugging in practice.
    *   The rubber duck as a symbol of clear communication and problem-solving within programming communities.
*   **Important Questions:**
    *   How does explaining code to a rubber duck help programmers identify errors?
    *   What are the psychological benefits of using a non-judgmental "listener" like a rubber duck for debugging?
    *   How has the rubber duck debugging technique spread within the programming community?
    *   What are the limitations of rubber duck debugging, and when is it not an effective solution?
*   **Specific Information:**
    *   **Programming Rubber Ducks:**
        *   Anecdotes and stories from programmers about their experiences with rubber duck debugging.
        *   References to rubber duck debugging in programming textbooks, online forums, and industry publications.
        *   Variations on the rubber duck debugging technique (e.g., using other inanimate objects).
    *   Interviews with programmers who use rubber duck debugging regularly.

**IV. The Rubber Duck as Art and Symbolism**

*   **Goal:** Analyze the use of rubber ducks as artistic mediums and their symbolic meanings in contemporary art and culture.
*   **Key Topics:**
    *   Florentijn Hofman's giant rubber duck installations and their global impact.
    *   The rubber duck as a symbol of peace, joy, and nostalgia.
    *   Other artists and designers who have incorporated rubber ducks into their work.
    *   The rubber duck as a statement on consumerism, globalization, and cultural identity.
*   **Important Questions:**
    *   What is the artistic significance of Hofman's giant rubber duck installations?
    *   How do audiences respond to the rubber duck as a public art piece?
    *   What are the different interpretations of the rubber duck's symbolism in contemporary art?
    *   How does the rubber duck challenge traditional notions of art and value?
*   **Specific Information:**
    *   **Giant Rubber Duck Art Installations:**
        *   Images and descriptions of Hofman's rubber duck installations in different locations around the world.
        *   Critical reviews and media coverage of the installations.
        *   Analysis of the cultural and political context of each installation.
    *   Examples of other artists and designers who have used rubber ducks in their work.
    *   Interviews with artists and critics about the rubber duck's artistic significance.

**V. The Rubber Duck in Commerce and Popular Culture**

*   **Goal:** Explore the commercial aspects of rubber ducks and their presence in various forms of media and entertainment.
*   **Key Topics:**
    *   The rubber duck toy market: size, growth, and key players.
    *   Rubber ducks as collectibles and novelty items.
    *   Rubber ducks in film, television, and literature.
    *   The use of rubber ducks in advertising and branding.
*   **Important Questions:**
    *   What are the key trends and challenges in the rubber duck toy market?
    *   How has the popularity of rubber ducks influenced their use in popular culture?
    *   What are the most memorable appearances of rubber ducks in film, television, and literature?
    *   How do companies use rubber ducks to promote their products and services?
*   **Important Data Points:**
    *   **Annual sales figures** for rubber ducks (globally and by region).
    *   Market share of leading rubber duck manufacturers.
    *   Growth rates of the rubber duck toy market.
    *   Data on the popularity of rubber duck collectibles (e.g., auction prices, online searches).
    *   Number of appearances of rubber ducks in film, television, and literature (quantifiable data).

**VI. Psychological and Sociological Impact**

*   **Goal:** Analyze the deeper psychological and sociological meanings behind the rubber duck's enduring appeal.
*   **Key Topics:**
    *   The rubber duck as a symbol of comfort, security, and nostalgia.
    *   The rubber duck's role in stress reduction and emotional well-being.
    *   The rubber duck as a cultural artifact reflecting societal values and trends.
    *   The rubber duck's ability to transcend cultural and linguistic barriers.
*   **Important Questions:**
    *   Why do people find comfort and security in rubber ducks?
    *   How can rubber ducks be used to promote emotional well-being?
    *   What do rubber ducks tell us about our society and culture?
    *   Why is the rubber duck such a universally recognized and appreciated symbol?
*   **Important Data Points:**
    *   **Psychological benefits of rubber duck debugging:** Studies or surveys on the perceived benefits of using rubber ducks for problem-solving, including stress reduction and improved focus.
    *   Surveys on people's emotional attachment to rubber ducks.
    *   Analysis of the cultural symbolism of rubber ducks in different countries and regions.

**VII. Conclusion**

*   **Goal:** Summarize the key findings of the research and offer concluding thoughts on the rubber duck's lasting cultural impact.
*   **Key Topics:**
    *   Recap of the rubber duck's journey from bath toy to cultural icon.
    *   Synthesis of the different aspects of the rubber duck's cultural impact (childhood, programming, art, commerce, psychology).
    *   Speculation on the future of the rubber duck in a rapidly changing world.
*   **Important Questions:**
    *   What are the most significant takeaways from this research?
    *   What is the rubber duck's most enduring contribution to society?
    *   How might the rubber duck's role in culture evolve in the years to come?

**VIII. Methodology**

*   **Document the methods used for the research.**
*   **Examples:** literature reviews, interviews, surveys, statistical analysis, historical analysis, art criticism.

**IX. Appendices (If Necessary)**

*   Supplementary data, interview transcripts, images, etc.

This research plan provides a comprehensive framework for your analysis report. Remember to cite your sources properly and to maintain a consistent and objective tone throughout your writing. Good luck!


Generating search queries...
Searching using 20 queries...
Searching (1/20): "history of rubber duck toys"
Searching (2/20): "rubber duck design evolution"
Searching (3/20): "rubber duck popular culture references"
Searching (4/20): "rubber duck bath time benefits"
Searching (5/20): "rubber duck sensory development infants"
Searching (6/20): "marketing rubber ducks children"
Searching (7/20): "rubber duck debugging origin"
Searching (8/20): "rubber duck debugging benefits programming"
Searching (9/20): "rubber duck debugging case studies"
Searching (10/20): "Florentijn Hofman rubber duck art significance"
Searching (11/20): "giant rubber duck installations public response"
Searching (12/20): "rubber duck symbolism contemporary art"
Searching (13/20): "rubber duck toy market size"
Searching (14/20): "rubber duck collectibles auction prices"
Searching (15/20): "rubber duck appearances film television"
Searching (16/20): "psychological benefits rubber duck comfort"
Searching (17/20): "rub

## Research Report: The cultural impact of rubber ducks in modern society

Okay, here's a comprehensive report on the cultural impact of rubber ducks in modern society, based on the provided research plan and search results. Due to the limited nature of the search results (mostly just links to Google searches without actual content), this report will be somewhat speculative and rely on general knowledge to fill in the gaps. Further research would be needed to provide a truly data-driven analysis.

**The Cultural Impact of Rubber Ducks in Modern Society**

**I. Introduction**

The rubber duck, a seemingly simple bath toy, has transcended its humble origins to become a multifaceted cultural icon. From its beginnings in the late 19th century as one of the first mass-produced rubber toys, the rubber duck has waddled its way into childhoods, programming practices, art installations, and the broader landscape of popular culture. This report will explore the rubber duck's journey, examining its impact on various aspects of modern society and analyzing the reasons behind its enduring appeal.

*   **Origins:** Early rubber toys, including ducks, emerged in the late 19th century with the advent of rubber manufacturing.
*   **Thesis:** The rubber duck's cultural significance lies in its ability to evoke nostalgia, facilitate problem-solving, serve as a canvas for artistic expression, and represent broader societal values.
*   **Report Structure:** This report will examine the rubber duck as a childhood staple, a programming tool, a form of art and symbolism, a commercial product, and a subject of psychological and sociological analysis.

**II. The Rubber Duck as a Childhood Staple**

The rubber duck's most traditional role is as a bath toy for children. Its bright color, simple shape, and often squeaky sound make it an engaging sensory experience for infants and toddlers.

*   **Bath Time Rituals:** Rubber ducks are often part of the bath time routine, providing a source of entertainment and distraction for children.
*   **Sensory Stimulation:** The texture, color, and sound of a rubber duck stimulate a child's senses, contributing to their development.
*   **Nostalgia:** For many adults, the rubber duck evokes fond memories of childhood bath times.
*   **Marketing:** Rubber ducks are heavily marketed towards parents and children, often featuring popular characters or themes.

    *   **Bathtub Rubber Ducks:**
        *   **Common Designs and Materials:** Typically made of PVC or rubber, often yellow, and designed to float upright.
        *   **Safety Standards and Regulations:** Subject to safety standards to prevent choking hazards and ensure non-toxicity.
        *   **Marketing Campaigns:** Often bundled with bath products or feature licensed characters.

**III. The Rubber Duck as a Programming Tool: Rubber Duck Debugging**

The rubber duck has found an unexpected role in the world of software development as a tool for debugging code.

*   **Origin:** "Rubber Duck Debugging" is a technique where a programmer explains their code, line by line, to a rubber duck (or any inanimate object).
*   **Benefits:** The act of explaining the code forces the programmer to think through the logic and often reveals errors or inconsistencies.
*   **Case Studies:** While specific case studies are not available in the search results, anecdotal evidence suggests that many programmers find this technique helpful.
*   **Symbolism:** The rubber duck symbolizes clear communication and a non-judgmental listener in the often-complex world of programming.

    *   **Programming Rubber Ducks:**
        *   **Anecdotes:** Programmers often share stories of how explaining code to a rubber duck helped them identify bugs.
        *   **References:** The technique is mentioned in programming books, online forums, and articles.

**IV. The Rubber Duck as Art and Symbolism**

The rubber duck has been embraced by artists as a medium for exploring themes of childhood, nostalgia, globalization, and consumerism.

*   **Florentijn Hofman's Installations:** Florentijn Hofman's giant rubber duck installations have traveled the world, bringing joy and sparking conversations about public art.
*   **Symbolism:** The rubber duck can symbolize peace, joy, and a return to simpler times.
*   **Other Artists:** Many other artists have incorporated rubber ducks into their work, using them to make statements about society and culture.

    *   **Giant Rubber Duck Art Installations:**
        *   **Images and Descriptions:** Hofman's installations are visually striking and have been featured in media around the world.
        *   **Critical Reviews:** The installations have received mixed reviews, with some praising their accessibility and others criticizing their lack of depth.
        *   **Cultural and Political Context:** The installations often take on different meanings depending on their location and the local context.

**V. The Rubber Duck in Commerce and Popular Culture**

The rubber duck is a commercially successful product, appearing in various forms of media and entertainment.

*   **Toy Market:** The rubber duck toy market is a significant segment of the overall toy industry.
*   **Collectibles:** Rubber ducks are popular collectibles, with some rare or unique ducks fetching high prices at auction.
*   **Film and Television:** Rubber ducks have made appearances in numerous films and television shows, often as a symbol of childhood or innocence.
*   **Advertising:** Companies use rubber ducks in advertising campaigns to evoke feelings of nostalgia and playfulness.

**VI. Psychological and Sociological Impact**

The rubber duck's enduring appeal can be attributed to its psychological and sociological significance.

*   **Comfort and Security:** The rubber duck can provide a sense of comfort and security, reminding people of simpler times.
*   **Stress Reduction:** Playing with or even just looking at a rubber duck can be a source of stress relief.
*   **Cultural Artifact:** The rubber duck reflects societal values and trends, evolving over time to reflect changing tastes and attitudes.
*   **Universality:** The rubber duck transcends cultural and linguistic barriers, making it a universally recognized and appreciated symbol.

**VII. Conclusion**

From its humble beginnings as a bath toy, the rubber duck has become a cultural icon with a multifaceted impact on modern society. Its ability to evoke nostalgia, facilitate problem-solving, serve as a canvas for artistic expression, and represent broader societal values has cemented its place in our collective consciousness. As the world continues to change, the rubber duck is likely to adapt and evolve, continuing to bring joy and spark conversations for generations to come.

**VIII. Methodology**

This report is based on a review of the provided search results and general knowledge about the cultural impact of rubber ducks. Due to the limited nature of the search results, further research would be needed to provide a more data-driven and comprehensive analysis.

**IX. Appendices**

(Not applicable due to the limited nature of the search results.)

**Further Research Needed:**

*   Market research data on the rubber duck toy market.
*   Studies on the psychological benefits of rubber duck debugging.
*   Critical analyses of Florentijn Hofman's rubber duck installations.
*   Surveys on people's emotional attachment to rubber ducks.
*   Data on the frequency of rubber duck appearances in film and television.


Report saved to 'the_cultural_impact_of_rubber_ducks_in_modern_soci_research_report.md'
